In [1]:
import geopandas as gpd
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [2]:
sites = pd.read_csv('global-inputs/sites.csv')
hr5_cells = gpd.read_file('global-inputs/HR5-change-cells-aoi.gpkg')

cells = hr5_cells[hr5_cells['index'].isin(sites['cell_id'])]

cells


,index,parent_id,sediment_area_ha,geometry
95,85da9213fffffff,84da921ffffffff,210.88,"POLYGON ((1528885.242 5425964.833, 1524656.417..."
128,85bb58c7fffffff,84bb58dffffffff,253.24,"POLYGON ((1595414.183 6163032.018, 1591203.235..."


In [3]:
shoreline_files = pd.Series(sorted(glob("data/HR5/*/shoreline_timeseries_tidal_correction.csv")))

shoreline_files

0    data/HR5/85bb58c7fffffff/shoreline_timeseries_...
1    data/HR5/85da9213fffffff/shoreline_timeseries_...
dtype: object

In [4]:
def calc_trends(file):
    df  = pd.read_csv(file)
    try:
        df.date = pd.to_datetime(df.date)
    except:
        print(file)
    df.index = (df.date - df.date.min()).dt.days / 365.25
    df.drop(columns=['date', 'Unnamed: 0', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg'], inplace=True, errors='ignore')
    trends = {}
    for col in df.columns:
        sub_df = df[col].dropna()
        
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1,1)
        y = sub_df
        linear_trend = LinearRegression().fit(x,y)
        pred = linear_trend.predict(x)

        trends['index'] = file.split('/')[2]
        if col == 'normalised_IW_shoreline_position':
            trends["IW shoreline trend"] = linear_trend.coef_[0]
            trends["IW shoreline intercept"] = linear_trend.intercept_
            trends["IW shoreline n_points"] = len(df[col])
            trends["IW shoreline n_points_nonan"] = len(sub_df)
            trends["IW shoreline r2_score"] = r2_score(y, pred)
            trends["IW shoreline mae"] = mean_absolute_error(y, pred)
            trends["IW shoreline mse"] = mean_squared_error(y, pred)
            trends["IW shoreline rmse"] = root_mean_squared_error(y, pred)
        else:
            trends["EOV shoreline trend"] = linear_trend.coef_[0]
            trends["EOV shoreline intercept"] = linear_trend.intercept_
            trends["EOV shoreline n_points"] = len(df[col])
            trends["EOV shoreline n_points_nonan"] = len(sub_df)
            trends["EOV shoreline r2_score"] = r2_score(y, pred)
            trends["EOV shoreline mae"] = mean_absolute_error(y, pred)
            trends["EOV shoreline mse"] = mean_squared_error(y, pred)
            trends["EOV shoreline rmse"] = root_mean_squared_error(y, pred)

    return pd.DataFrame([trends])
    

In [5]:
trends = pd.concat([calc_trends(f) for f in shoreline_files])

trends

test shell script - notebook 2


In [6]:
trends_file_path = 'shoreline_trends_by_cell.geojson'
# if os.path.exists(trends_file_path):
#     cell_trends = gpd.read_file('shoreline_trends_by_cell.geojson')
#     cell_trends.to_crs("EPSG:4326", inplace=True)
#     cell_trends.update(trends)
#     cell_trends.to_file(trends_file_path)
# else:
cell_trends = cells.merge(trends, on='index')
cell_trends.to_crs("EPSG:4326", inplace=True)
cell_trends.to_file(trends_file_path)
    

cell_trends